# Managing Products
The Descartes Labs Catalog API is a single interface through which you can discover existing raster datasets, search and retrieve their associated images, and manage your own datasets. Here we will outline the basic steps to create a new raster product with associated imagery. This example will also cover the basics of namespaces and organizations for managing access to your newly created Product.

This guide is meant to serve as an introduction to searching and retrieving raster data. For a more in depth overview of all Catalog classes and their capabilities please visit the [API Reference](https://docs.descarteslabs.com/descarteslabs/catalog/readme.html) and [Catalog Guide](https://docs.descarteslabs.com/guides/catalog.html) sections in our Documentation page.

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import (
    Image,
    DataType,
    Product,
    Resolution,
    ResolutionUnit,
    SpectralBand,
)

## Creating and updating a Product
The only required attributes for a Product are a *unique* id and a name. 

Product IDs always prepend your organization upon creation:

    my-org-id:my-product-id

You can retrieve your organization by the [`Auth`](https://docs.descarteslabs.com/descarteslabs/auth/readme.html) API:

In [ ]:
org = dl.auth.Auth().payload["org"]
org

Since you may have other users within your organization who have run this same example, let's append our user namespace to our Product ID. Each Descartes Labs account has a unique namespace ID:

In [ ]:
user_namespace = dl.auth.Auth().namespace
user_namespace

_Note: You do not always need to pass your personal namespace in practice, make your Product ID whatever you'd like (as long as it's unique!)_

In [ ]:
pid = f"sample-product:{user_namespace}"
pid

We can create our new dataset by calling [`Product.get_or_create()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Product.get_or_create), giving it a name, [`tags`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Product.tags), and importantly saving everything through [`Product.save()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Product.save):

In [ ]:
product = Product.get_or_create(pid)
product.name = "Example RGB Product"
product.tags = ["examples"]
product.save()
product

## Creating bands
Before adding any imagery to a Product you must [`Band`s](https://docs.descarteslabs.com/descarteslabs/catalog/docs/band.html) that declare the structure of the data shared among all images in a product.

We'll start with a [`SpectralBand`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/band.html#descarteslabs.catalog.SpectralBand), which defines data that lie somewhere on the electro-opitcal spectrum. Here we have a GeoTIFF located under the path *"data/rgb.tif"*  which contains 3 spectral bands. We will create one band per channel here, for __red__, __green__, and __blue__, including supported wavelength properties:

In [ ]:
# These values are in nanometers (nm)
band_info_dict = {
    "red": {"wavelength_nm_min": 650, "wavelength_nm_max": 680},
    "green": {"wavelength_nm_min": 542.5, "wavelength_nm_max": 577.5},
    "blue": {"wavelength_nm_min": 457.5, "wavelength_nm_max": 522.5},
}

A band defines where its data is found in the files attached to the images associated to the product, uniquely identified by its name and Product ID. 

The full id of the band is composed of the Product ID and the name:

In [ ]:
for i, (name, info) in enumerate(band_info_dict.items()):
    band = SpectralBand(name=name, product=product)
    band.band_index = i
    band.file_index = 0
    band.data_type = DataType.FLOAT64
    band.data_range = (0, 1)
    band.display_range = (0, 0.4)
    band.resolution = Resolution(unit=ResolutionUnit.METERS, value=30.0)
    band.wavelength_nm_min = info["wavelength_nm_min"]
    band.wavelength_nm_max = info["wavelength_nm_max"]
    band.save()
    print(f"Saved: {band.id}")

In the above example:
* *band_index = 0* indicates that __red__ is the first band in the source file
* *file_index = 0* indicates we expect the first source file passed to contain this band's data 
* Each band is expected to be represented by 64-bit floating point ([`DataType.FLOAT64`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/types.html#descarteslabs.catalog.DataType))
* *data_range* and *display_range* are the default from [Sentinel-2 L2A](https://app.descarteslabs.com/explorer/datasets/esa:sentinel-2:l2a:v1)
* Each band is expected to have a [`Resolution`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/types.html#descarteslabs.catalog.Resolution) of 30 meters

Note also that if we were to set *file_index = 1* it would expect a second Image in that position to represent that band. Since all 3 bands are in the same file, we kept that at 0.

See the [`Band` Documentation page](https://docs.descarteslabs.com/descarteslabs/catalog/docs/band.html#bands) for more information about properties and other band types not covered in this example.

## Uploading Images - File Paths
[`Image.upload()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.Image.upload) uploads imagery to a Product via a list of specified file paths. This returns an [`ImageUpload`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.ImageUpload) object. Images are uploaded and processed asynchronously, so they are not available in the catalog immediately. With [`ImageUpload.wait_for_completion()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.ImageUpload.wait_for_completion) we wait until the upload is completely finished:

In [ ]:
# Set any attributes that should be set on the uploaded images
image = Image(product=product, name="image1")
# Set acquired date
image.acquired = "2023-06-15"
# Set metadata
image.cloud_fraction = 0.1
# Do the upload
image_path = "data/rgb.tif"
upload = image.upload(image_path)
upload.wait_for_completion()
upload.status

### _Note: Check your Results!_
You can now search and retrieve this imagery in either [Explorer](https://app.descarteslabs.com/explorer) or Catalog API directly. We'll retrieve this image's data directly through the [`Image.ndarray()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.Image.ndarray) method:

In [ ]:
ndarr = image.ndarray(bands=["red", "green", "blue"])
dl.utils.display(ndarr, size=5)

## Uploading Images - Numpy Arrays
You can also call [`Image.upload_ndarray()`](https://docs.descarteslabs.com/examples-gallery/plot_create_product.html?#upload-ndarray-to-new-product) if you have your pixel data in-memory and not in a static file. Note you will need to manually specify raster metadata as that is not information contained within a numpy array.

## Access Control
By default only the creator of a Product can read and modify it as well as read and modify the bands and imagery in it. To share access to a Product with others you can modify its access control lists (ACLs):

In [ ]:
product.readers = ["org:pga-tour"]
product.writers = ["email:jane.doe@descarteslabs.com", "email:john.daly@gmail.com"]
product.save()

In the above example:
* All users in the "pga-tour" organization can now find and retrieve data from this Product
* Two specific users identified by email can now update the Product and add new imagery

### Transfer ownership
Transferring ownership of a product to a new user requires cooperation from both the previous owner and the new owner and is a two-step effort. The first step is for the previous owner to add the new owner to the product:

    product.owners.append(f"user:{user_namespace}")
    product.save()

Just a reminder that you cannot use the "email:" variant as an owner. You will have to request the user's default namespace ID from the new owner and use that instead. You can find your default Namespace ID in the profile drop-down on [iam.descarteslabs.com](iam.descarteslabs.com) or through [`dl.auth.Auth().namespace`](https://docs.descarteslabs.com/descarteslabs/auth/readme.html#descarteslabs.auth.Auth.namespace).

The second step is for the new owner to remove the previous owner:

    product.owners.remove(f"user:{user_namespace}")
    product.save()


For further information on access control please see the [Sharing Resources](https://docs.descarteslabs.com/guides/sharing.html) guide.

## Deleting Bands and Products
We can delete our Product through the [`Product.delete()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Product.delete) method:

In [ ]:
product.delete()

## *NOTE:* You ran into an error, didn't you!
A Product can _only be deleted if it doesn’t have any associated bands or images_. Because the Product we created still has images and bands we must first delete those related objects. 

There is a convenience method [`Product.delete_related_objects()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Product.delete_related_objects) to delete all bands and images in a product. Be careful as this may delete a lot of data and can’t be undone!

In [ ]:
status = product.delete_related_objects()

This kicks off a job that deletes bands and images in the background. You can wait for this to complete and then delete the product:

In [ ]:
if status:
    status.wait_for_completion()
product.delete()